In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [30]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [31]:
X = data.drop('MEDV',axis=1)
y = data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
### Ваш код ###

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [110]:
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
### Ваш код ###
%time
#линейная регрессия
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred)
mse_lr = mean_squared_error(y_test, y_pred)
print('r2_lr:', r2_lr, 'MSE_lr = ', mse_lr)

#Ridge
ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred_r = ridge.predict(X_test)
r2_ridge = r2_score(y_test, y_pred_r)
mse_ridge = mean_squared_error(y_test, y_pred_r)
print('r2_ridge:', r2_ridge, 'MSE_ridge = ', mse_ridge)

#Lasso
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred_l = lasso.predict(X_test)
r2_lasso = r2_score(y_test, y_pred_l)
mse_lasso = mean_squared_error(y_test, y_pred_l)
print('r2_lasso:', r2_lasso, 'MSE_lasso = ', mse_lasso)

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

Wall time: 0 ns
r2_lr: 0.668482575397157 MSE_lr =  24.311426929693596
r2_ridge: 0.6659608075261688 MSE_ridge =  24.49635770792532
r2_lasso: 0.6668687223368214 MSE_lasso =  24.42977688007288


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

In [29]:
parametrs = { 'alpha': np.array([10 ** n for n in range(-5, 6)])}
dip = np.array([10 ** n for n in range(-5, 6)])
dip

array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])

In [47]:
### Ваш код ###
#Ridge_GridSearchCV
gs_r = GridSearchCV(Ridge(), parametrs).fit(X, y)
y_pred_gs_r = gs_r.predict(X_test)
r2_ridge_grid_search = r2_score(y_test, y_pred_gs_r)
print('r2_ridge_grid_search:', r2_ridge_grid_search)

#Lasso_GridSearchCV
gs_l = GridSearchCV(Lasso(), parametrs).fit(X, y)
y_pred_gs_l = gs_l.predict(X_test)
r2_lasso_grid_search = r2_score(y_test, y_pred_gs_l)
print('r2_lasso_grid_search:', r2_lasso_grid_search)

#RidgeCV
RCV = RidgeCV(alphas = dip).fit(X, y)
y_pred_rcv = RCV.predict(X_test)
r2_ridge_cv = r2_score(y_test, y_pred_rcv)
print('r2_ridge_cv:', r2_ridge_cv)

#LassoCV
LCV = LassoCV(alphas = dip, random_state=42).fit(X, y)
y_pred_lcv = LCV.predict(X_test)
r2_lasso_cv = r2_score(y_test, y_pred_lcv)
print('r2_lasso_cv:', r2_lasso_cv)

# r2_ridge_grid_search = 0
# r2_ridge_cv = 0
# r2_lasso_grid_search = 0
# r2_lasso_cv = 0
results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

r2_ridge_grid_search: 0.7021488961592373
r2_lasso_grid_search: 0.6919024192156511
r2_ridge_cv: 0.7024461750553552
r2_lasso_cv: 0.6883086259341868


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [20]:
# StandardScaler
scaler = StandardScaler()
X_trn_scaled = scaler.fit_transform(X_train)
X_tst_scaled = scaler.transform(X_test)

# MinMaxScaler
min_max_scaler = MinMaxScaler()
X_trn_scaled_mm = min_max_scaler.fit_transform(X_train)
X_tst_scaled_mm = min_max_scaler.transform(X_test)

In [39]:
#Ridge_sts
ridge = Ridge()
ridge.fit(X_trn_scaled, y_train)
y_pred_sts = ridge.predict(X_tst_scaled)
r2_ridge_sts = r2_score(y_test, y_pred_sts)
mse_ridge_sts = mean_squared_error(y_test, y_pred_sts)
print('r2_ridge_sts:', r2_ridge_sts, 'MSE_ridge_sts = ', mse_ridge_sts)

#Ridge_mms
# ridge = Ridge()
ridge.fit(X_trn_scaled_mm, y_train)
y_pred_mms = ridge.predict(X_tst_scaled_mm)
r2_ridge_mms = r2_score(y_test, y_pred_mms)
mse_ridge_mms = mean_squared_error(y_test, y_pred_mms)
print('r2_ridge_mms:', r2_ridge_mms, 'MSE_ridge_mms = ', mse_ridge_mms)

#Lasso_sts
lasso_s = Lasso()
lasso_s.fit(X_trn_scaled, y_train)
y_pred_sts = lasso_s.predict(X_tst_scaled)
r2_lasso_sts = r2_score(y_test, y_pred_sts)
mse_lasso_sts = mean_squared_error(y_test, y_pred_sts)
print('r2_lasso_sts:', r2_lasso_sts, 'MSE_lasso = ', mse_lasso_sts)

#Lasso
lasso_m = Lasso()
lasso_m.fit(X_trn_scaled_mm, y_train)
y_pred_mms = lasso_m.predict(X_tst_scaled_mm)
r2_lasso_mms = r2_score(y_test, y_pred_mms)
mse_lasso_mms = mean_squared_error(y_test, y_pred_mms)
print('r2_lasso_mms:', r2_lasso_mms, 'MSE_lasso = ', mse_lasso_mms)

r2_ridge_sts: 0.6681901076774428 MSE_ridge_sts =  24.332874694032522
r2_ridge_mms: 0.6762207658974595 MSE_ridge_mms =  23.743956145491147
r2_lasso_sts: 0.624044752347846 MSE_lasso =  27.57022060930858
r2_lasso_mms: 0.2573921442545195 MSE_lasso =  54.45824346639145


In [22]:
from sklearn.pipeline import Pipeline

In [41]:
pipe_sts = Pipeline(steps=[
    ('st_scaler', StandardScaler()),
    ('ridge', Ridge())
])
pipe_sts.fit(X_train, y_train)
y_pred_sts = pipe_sts.predict(X_test)
r2_ridge_sts = r2_score(y_test, y_pred_sts)
mse_ridge_sts = mean_squared_error(y_test, y_pred_sts)
print('r2_ridge_sts:', r2_ridge_sts, 'MSE_ridge_sts = ', mse_ridge_sts)

pipe_mms = Pipeline(steps=[
    ('mm_scaler', MinMaxScaler()),
    ('ridge', Ridge())
])
pipe_mms.fit(X_train, y_train)
y_pred_mms = pipe_mms.predict(X_test)
r2_ridge_mms = r2_score(y_test, y_pred_mms)
mse_ridge_mms = mean_squared_error(y_test, y_pred_mms)
print('r2_ridge_mms:', r2_ridge_mms, 'MSE_ridge_mms = ', mse_ridge_mms)

pipe_lsts = Pipeline(steps=[
     ('st_scaler', StandardScaler()),
     ('lasso', Lasso())
])
pipe_lsts.fit(X_train, y_train)
y_pred_lsts = pipe_lsts.predict(X_test)
r2_lasso_sts = r2_score(y_test, y_pred_lsts)
mse_lasso_sts = mean_squared_error(y_test, y_pred_lsts)
print('r2_lasso_sts:', r2_lasso_sts, 'MSE_lasso_sts = ', mse_lasso_sts)

pipe_lmms = Pipeline(steps=[
    ('mm_scaler', MinMaxScaler()),
    ('lasso', Lasso())
])
pipe_lmms.fit(X_train, y_train)
y_pred_lmms = pipe_lmms.predict(X_test)
r2_lasso_mms = r2_score(y_test, y_pred_lmms)
mse_lasso_mms = mean_squared_error(y_test, y_pred_lmms)
print('r2_lasso_mms:', r2_lasso_mms, 'MSE_lasso_mms = ', mse_lasso_mms)

### Ваш код ###
r2_ridge_standart_scaler = r2_ridge_sts
r2_ridge_min_max_scaler = r2_ridge_mms
r2_lasso_standart_scaler = r2_lasso_sts
r2_lasso_min_max_scaler = r2_lasso_mms
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]


r2_ridge_sts: 0.6681901076774428 MSE_ridge_sts =  24.332874694032522
r2_ridge_mms: 0.6762207658974595 MSE_ridge_mms =  23.743956145491147
r2_lasso_sts: 0.624044752347846 MSE_lasso_sts =  27.57022060930858
r2_lasso_mms: 0.2573921442545195 MSE_lasso_mms =  54.45824346639145


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [56]:
### Ваш код ###
# # StandardScaler
# X_trn_scaled = scaler.fit_transform(X_train)
# X_tst_scaled = scaler.transform(X_test)
# # MinMaxScaler
# X_trn_scaled_mm = min_max_scaler.fit_transform(X_train)
# X_tst_scaled_mm = min_max_scaler.transform(X_test)

#RidgeCV_StandardScaler
rcv_sts = RidgeCV(alphas = dip).fit(X_trn_scaled, y_train)
y_pred_rcv_sts = rcv_sts.predict(X_tst_scaled)
r2_ridge_standart_scaler_cv = r2_score(y_test, y_pred_rcv_sts)
print('r2_ridge_cv_sts:', r2_ridge_standart_scaler_cv)

#LassoCV_StandardScaler
lcv_sts = LassoCV(alphas = dip, cv=5, random_state=42).fit(X_trn_scaled, y_train)
y_pred_lcv_sts = lcv_sts.predict(X_tst_scaled)
r2_lasso_standart_scaler_cv = r2_score(y_test, y_pred_lcv_sts)
print('r2_lasso_cv_sts:', r2_lasso_standart_scaler_cv)

#RidgeCV_MinMaxScaler
rcv_sts = RidgeCV(alphas = dip).fit(X_trn_scaled_mm, y_train)
y_pred_rcv_mms = rcv_sts.predict(X_tst_scaled_mm)
r2_ridge_min_max_scaler_cv = r2_score(y_test, y_pred_rcv_mms)
print('r2_ridge_cv_mms:', r2_ridge_min_max_scaler_cv)

#LassoCV_MinMaxScaler
lcv_sts = LassoCV(alphas = dip, cv=5, random_state=42).fit(X_trn_scaled_mm, y_train)
y_pred_lcv_mms = lcv_sts.predict(X_tst_scaled_mm)
r2_lasso_min_max_scaler_cv = r2_score(y_test, y_pred_lcv_mms)
print('r2_lasso_cv_mms:', r2_lasso_min_max_scaler_cv)

# r2_ridge_standart_scaler_cv = 0
# r2_ridge_min_max_scaler_cv = 0
# r2_lasso_standart_scaler_cv = 0
# r2_lasso_min_max_scaler_cv = 0
results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]
results_regression.loc[(results_regression['task'] == 'task4') | (results_regression['task'] == 'task5')]

r2_ridge_cv_sts: 0.665730534061687
r2_lasso_cv_sts: 0.6684821312777706
r2_ridge_cv_mms: 0.6697651112858509
r2_lasso_cv_mms: 0.6684836564428189


,model,task,R2
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045
10,Lasso_MinMaxScaler,task4,0.257392
11,Ridge_StandardScaler_CV,task5,0.665731
12,Ridge_MinMaxScaler_CV,task5,0.669765
13,Lasso_StandardScaler_CV,task5,0.668482
14,Lasso_MinMaxScaler_CV,task5,0.668484


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [57]:
from sklearn.preprocessing import PolynomialFeatures

In [66]:
### Ваш код ###
poly = PolynomialFeatures(2)
# # StandardScaler
# X_trn_scaled = scaler.fit_transform(X_train)
# X_tst_scaled = scaler.transform(X_test)
X_trn_sscaled_poly = poly.fit_transform(X_trn_scaled)
X_tst_sscaled_poly = poly.transform(X_tst_scaled)

# # MinMaxScaler
# X_trn_scaled_mm = min_max_scaler.fit_transform(X_train)
# X_tst_scaled_mm = min_max_scaler.transform(X_test)
X_trn_sscaled_mm_poly = poly.fit_transform(X_trn_scaled_mm)
X_tst_sscaled_mm_poly = poly.transform(X_tst_scaled_mm)

# r2_ridge_standart_scaler_poly
ridge_pf_st = Ridge()
ridge_pf_st.fit(X_trn_sscaled_poly, y_train)
y_pred_r_sts = ridge_pf_st.predict(X_tst_sscaled_poly)
r2_ridge_standart_scaler_poly = r2_score(y_test, y_pred_r_sts)
print('r2_ridge_standart_scaler_poly:', r2_ridge_standart_scaler_poly)

# r2_ridge_min_max_scaler_poly
ridge_pf_mm = Ridge()
ridge_pf_mm.fit(X_trn_sscaled_mm_poly, y_train)
y_pred_r_mms = ridge_pf_mm.predict(X_tst_sscaled_mm_poly)
r2_ridge_min_max_scaler_poly = r2_score(y_test, y_pred_r_mms)
print('r2_ridge_min_max_scaler_poly:', r2_ridge_min_max_scaler_poly)

# r2_lasso_standart_scaler_poly
lasso_pf_st = Lasso()
lasso_pf_st.fit(X_trn_sscaled_poly, y_train)
y_pred_l_sts = lasso_pf_st.predict(X_tst_sscaled_poly)
r2_lasso_standart_scaler_poly = r2_score(y_test, y_pred_l_sts)
print('r2_lasso_standart_scaler_poly:', r2_lasso_standart_scaler_poly)

# r2_lasso_min_max_scaler_poly
lasso_pf_mm = Lasso()
lasso_pf_mm.fit(X_trn_sscaled_mm_poly, y_train)
y_pred_l_mms = lasso_pf_mm.predict(X_tst_sscaled_mm_poly)
r2_lasso_min_max_scaler_poly = r2_score(y_test, y_pred_l_mms)
print('r2_lasso_min_max_scaler_poly:', r2_lasso_min_max_scaler_poly)

# r2_ridge_standart_scaler_poly = 0
# r2_ridge_min_max_scaler_poly = 0
# r2_lasso_standart_scaler_poly = 0
# r2_lasso_min_max_scaler_poly = 0
results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
results_regression.loc[(results_regression['task'] == 'task5') | (results_regression['task'] == 'task6')]

r2_ridge_standart_scaler_poly: 0.8171359789657088
r2_ridge_min_max_scaler_poly: 0.8298862467502746
r2_lasso_standart_scaler_poly: 0.732273828270899
r2_lasso_min_max_scaler_poly: 0.2611262741735658


,model,task,R2
11,Ridge_StandardScaler_CV,task5,0.665731
12,Ridge_MinMaxScaler_CV,task5,0.669765
13,Lasso_StandardScaler_CV,task5,0.668482
14,Lasso_MinMaxScaler_CV,task5,0.668484
15,Ridge_StandardScaler_Poly,task6,0.817136
16,Ridge_MinMaxScaler_Poly,task6,0.829886
17,Lasso_StandardScaler_Poly,task6,0.732274
18,Lasso_MinMaxScaler_Poly,task6,0.261126


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [68]:
### Ваш код ###

# r2_ridge_standart_scaler_poly_cv
ridge_sts_cv_poly = RidgeCV(alphas = dip).fit(X_trn_sscaled_poly, y_train)
y_pred_r_sts_cv_poly = ridge_sts_cv_poly.predict(X_tst_sscaled_poly)
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, y_pred_r_sts_cv_poly)
print('r2_ridge_standart_scaler_poly_cv:', r2_ridge_standart_scaler_poly_cv)

# r2_ridge_min_max_scaler_poly_cv
ridge_mms_cv_poly = RidgeCV(alphas = dip).fit(X_trn_sscaled_mm_poly, y_train)
y_pred_r_mms_cv_poly = ridge_mms_cv_poly.predict(X_tst_sscaled_mm_poly)
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, y_pred_r_mms_cv_poly)
print('r2_ridge_min_max_scaler_poly_cv:', r2_ridge_min_max_scaler_poly_cv)

# r2_lasso_standart_scaler_poly_cv
lasso_sts_cv_poly = LassoCV(alphas = dip, cv=5, random_state=42).fit(X_trn_sscaled_poly, y_train)
y_pred_l_sts_cv_poly = lasso_sts_cv_poly.predict(X_tst_sscaled_poly)
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, y_pred_l_sts_cv_poly)
print('r2_lasso_standart_scaler_poly_cv:', r2_lasso_standart_scaler_poly_cv)

# r2_lasso_min_max_scaler_poly_cv 
lasso_mms_cv_poly = LassoCV(alphas = dip, cv=5, random_state=42).fit(X_trn_sscaled_mm_poly, y_train)
y_pred_l_mms_cv_poly = lasso_mms_cv_poly.predict(X_tst_sscaled_mm_poly)
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, y_pred_l_mms_cv_poly)
print('r2_lasso_min_max_scaler_poly_cv:', r2_lasso_min_max_scaler_poly_cv)

# r2_ridge_standart_scaler_poly_cv = 0
# r2_ridge_min_max_scaler_poly_cv = 0
# r2_lasso_standart_scaler_poly_cv = 0
# r2_lasso_min_max_scaler_poly_cv = 0
results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
results_regression.loc[(results_regression['task'] == 'task6') | (results_regression['task'] == 'task7')]

r2_ridge_standart_scaler_poly_cv: 0.8187344606117026
r2_ridge_min_max_scaler_poly_cv: 0.8501402125960564
r2_lasso_standart_scaler_poly_cv: 0.8128138856150267
r2_lasso_min_max_scaler_poly_cv: 0.8397022991682292


,model,task,R2
15,Ridge_StandardScaler_Poly,task6,0.817136
16,Ridge_MinMaxScaler_Poly,task6,0.829886
17,Lasso_StandardScaler_Poly,task6,0.732274
18,Lasso_MinMaxScaler_Poly,task6,0.261126
19,Ridge_StandardScaler_Poly_CV,task7,0.818734
20,Ridge_MinMaxScaler_Poly_CV,task7,0.850140
21,Lasso_StandardScaler_Poly_CV,task7,0.812814
22,Lasso_MinMaxScaler_Poly_CV,task7,0.839702


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [126]:
### Ваш код ##
best_params = {}
r2 = 0
ridge = Ridge()
lasso = Lasso() # max_iter=10000
st_scaler = StandardScaler()
mm_scaler = MinMaxScaler()
poly_features = PolynomialFeatures()

steps = [('poly_features', poly_features), ('minmax', mm_scaler), ('ridge', ridge)]
params = {'ridge__alpha': [10 ** x for x in range(-5, 6)], 'poly_features__degree': [x for x in range(1, 6)]}
grid = GridSearchCV(Pipeline(steps), cv=5, param_grid=params).fit(X_train, y_train)
best_score = grid.best_score_
r2_score = grid.score(X_test, y_test)
ridge_alpha = grid.best_estimator_.get_params()['ridge__alpha']
poly_features_degree = grid.best_estimator_.get_params()['poly_features__degree']
r2_best_model = r2_score
best_params = grid.best_estimator_.get_params()

steps = [('poly_features', poly_features), ('stand', st_scaler), ('ridge', ridge)]
params = {'ridge__alpha': [10 ** x for x in range(-5, 6)], 'poly_features__degree': [x for x in range(1, 6)]}
grid = GridSearchCV(Pipeline(steps), cv=5, param_grid=params).fit(X_train, y_train)
best_score = grid.best_score_
r2_score = grid.score(X_test, y_test)
ridge_alpha = grid.best_estimator_.get_params()['ridge__alpha']
poly_features_degree = grid.best_estimator_.get_params()['poly_features__degree']
if r2_best_model < r2_score :
    best_params = grid.best_estimator_.get_params()
    r2_best_model = r2_score
    
steps = [('poly_features', poly_features), ('minmax', mm_scaler), ('lasso', lasso)]
params = {'lasso__alpha': [10 ** x for x in range(-5, 6)], 'poly_features__degree': [x for x in range(1, 6)]}
grid = GridSearchCV(Pipeline(steps), cv=5, param_grid=params).fit(X_train, y_train)
best_score = grid.best_score_
r2_score = grid.score(X_test, y_test)
lasso_alpha = grid.best_estimator_.get_params()['lasso__alpha']
poly_features_degree = grid.best_estimator_.get_params()['poly_features__degree']
if r2_best_model < r2_score :
    best_params = grid.best_estimator_.get_params()
    r2_best_model = r2_score

steps = [('poly_features', poly_features), ('stand', st_scaler), ('lasso', lasso)]
params = {'lasso__alpha': [10 ** x for x in range(-5, 6)], 'poly_features__degree': [x for x in range(1, 6)]}
grid = GridSearchCV(Pipeline(steps), cv=5, param_grid=params).fit(X_train, y_train)
best_score = grid.best_score_
r2_score = grid.score(X_test, y_test)
lasso_alpha = grid.best_estimator_.get_params()['lasso__alpha']
poly_features_degree = grid.best_estimator_.get_params()['poly_features__degree']
if r2_best_model < r2_score :
    best_params = grid.best_estimator_.get_params()
    r2_best_model = r2_score

print('r2_best_model : ', r2_best_model)
print('Параметры лучшей модели:\n', best_params)
# r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

r2_best_model :  0.8661932037579262
Параметры лучшей модели:
 {'memory': None, 'steps': [('poly_features', PolynomialFeatures(degree=3)), ('minmax', MinMaxScaler()), ('ridge', Ridge(alpha=0.1))], 'verbose': False, 'poly_features': PolynomialFeatures(degree=3), 'minmax': MinMaxScaler(), 'ridge': Ridge(alpha=0.1), 'poly_features__degree': 3, 'poly_features__include_bias': True, 'poly_features__interaction_only': False, 'poly_features__order': 'C', 'minmax__clip': False, 'minmax__copy': True, 'minmax__feature_range': (0, 1), 'ridge__alpha': 0.1, 'ridge__copy_X': True, 'ridge__fit_intercept': True, 'ridge__max_iter': None, 'ridge__normalize': False, 'ridge__random_state': None, 'ridge__solver': 'auto', 'ridge__tol': 0.001}


In [25]:
results_regression.sort_values(by='R2', ascending = False). head(1)

,model,task,R2


http://archive.ics.uci.edu/ml/datasets/Adult

In [4]:
data2 = pd.read_csv('adult.csv')
data2.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [5]:
### Ваш код ###
data2.loc[data2["class"] == "<=50K", "class"] = 1
data2.loc[data2["class"] == ">50K", "class"] = 0
data2["class"] = pd.to_numeric(data2["class"])

In [6]:
X2 = data2.drop('class',axis=1)
y2 = data2['class'].to_frame()
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((39073, 14), (9769, 14), (39073, 1), (9769, 1))

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [7]:
data2['class'].value_counts()

1    37155
0    11687
Name: class, dtype: int64

In [8]:
### Ваш код ###
# метрика accuracy
acc = 37155/48842
# По условию в целевой переменной предсказали только значение 1.
TP = 37155
FP = 48842 - 37155
# 𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛 = 𝑇𝑃/(𝑇𝑃 + 𝐹𝑃)
𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛 = TP/(TP + FP)

# По условию в целевой переменной значение 0 не предсказывали, поэтому значения FN и TN равны 0.
# recall = TP/(TP + FN)
recall = 1
f1_score = 2*(precision * recall)/(precision + recall)

print('acc:',acc)
print('f1_score :', f1_score)
f1_most_frequent = f1_score
acc_most_frequent = acc
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

acc: 0.7607182343065395
f1_score : 0.8640999104619929


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [95]:
### Ваш код ###
# data_a.info()
data2.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [41]:
### Ваш код ###
digit_var = [x for x in data2.columns if data2[x].dtype != 'object']
categ_var = [x for x in data2.columns if data2[x].dtype == 'object']
print(digit_var)
print(categ_var)

['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'class']
['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [64]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC

In [57]:
X2 = data2.drop('class',axis=1)
y2 = data2['class'].to_frame()
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
num_cols = [x for x in X2.columns if X2[x].dtype != 'object']
cat_cols = [x for x in X2.columns if X2[x].dtype == 'object']

num_pipeline = Pipeline(steps=[('scale',MinMaxScaler())])
# cat_pipeline = Pipeline(steps=[('le',LabelEncoder())])
cat_pipeline = Pipeline(steps=[('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

# LogisticRegression
clf = LogisticRegression(random_state=0)
clf_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', clf)
])
lr = clf_pipeline.fit(X2_train, y2_train)
f1_LR = cross_val_score(lr, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_LR:' ,f1_LR)
acc_LR = cross_val_score(lr, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_LR:' ,acc_LR)

# KNeighborsClassifier
knnf = KNeighborsClassifier(n_neighbors=2)
knn_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', knnf)
])
knn = knn_pipeline.fit(X2_train, y2_train)
f1_KNN = cross_val_score(knn, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_KNN:' , f1_KNN)
acc_KNN = cross_val_score(knn, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_KNN:', acc_KNN)

# LinearSVC
svmf = LinearSVC(random_state=0, tol=1e-5)
svm_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', svmf)
])
svm = svm_pipeline.fit(X2_train, y2_train)
f1_SVM = cross_val_score(svm, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_SVM:' ,f1_SVM)
acc_SVM = cross_val_score(svm, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_SVM:' ,acc_SVM)

# f1_LR = 0
# acc_LR = 0
# f1_KNN = 0
# acc_KNN = 0`
# f1_SVM = 0
# acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

f1_LR: 0.9043543031430362
acc_LR: 0.8499730484279429
f1_KNN: 0.8423445665914194
acc_KNN: 0.7758553744374281
f1_SVM: 0.9060887807197108
acc_SVM: 0.8524555994661561


In [12]:
from sklearn import set_config
set_config(display='diagram')
display(clf_pipeline)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('model', LogisticRegression(random_state=0))])

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [13]:
from sklearn.impute import SimpleImputer

In [21]:
num_pipe = Pipeline(steps=[('scale',MinMaxScaler())])
cat_pipe = Pipeline(steps=[('Simpl',SimpleImputer(missing_values='?', strategy= 'most_frequent')),
                           ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipe,num_cols),
    ('cat_pipeline',cat_pipe,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

# LogisticRegression
clfs = LogisticRegression(random_state=0)
clfs_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', clfs)
])
lr = clfs_pipeline.fit(X2_train, y2_train)
f1_LR = cross_val_score(lr, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_LR:' ,f1_LR)
acc_LR = cross_val_score(lr, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_LR:' ,acc_LR)

# KNeighborsClassifier
knnfs = KNeighborsClassifier(n_neighbors=2)
knns_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', knnfs)
])
knn = knns_pipeline.fit(X2_train, y2_train)
f1_KNN = cross_val_score(knn, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_KNN:' , f1_KNN)
acc_KNN = cross_val_score(knn, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_KNN:', acc_KNN)

# LinearSVC
svmfs = LinearSVC(random_state=0, tol=1e-5)
svms_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', svmfs)
])
svm = svms_pipeline.fit(X2_train, y2_train)
f1_SVM = cross_val_score(svm, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_SVM:' ,f1_SVM)
acc_SVM = cross_val_score(svm, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_SVM:' ,acc_SVM)

# f1_LR = 0
# acc_LR = 0
# f1_KNN = 0
# acc_KNN = 0
# f1_SVM = 0
# acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

f1_LR: 0.9046146622492456
acc_LR: 0.8502034258128308
f1_KNN: 0.8414260520878266
acc_KNN: 0.7745245413359617
f1_SVM: 0.9055890262100279
acc_SVM: 0.8514574736575696


In [17]:
from sklearn import set_config
set_config(display='diagram')
display(clfs_pipeline)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('scale',
                                                                   MinMaxScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('Simpl',
                                                                   SimpleImputer(missing_values='?',
                                                                                 strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('model', LogisticRegression(random_state=0))])

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [34]:
data3=data2.copy()
data3[data3.iloc[:,:] == '?'].count()

age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
class                0
dtype: int64

In [35]:
data3.loc[data3['workclass'] == '?', 'workclass'] = ''
data3.loc[data3['occupation'] == '?', 'occupation'] = ''
data3.loc[data3['native-country'] == '?', 'native-country'] = ''
data3[data3.iloc[:,:] == '?'].count()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [36]:
X3 = data3.drop('class',axis=1)
y3 = data3['class'].to_frame()
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=42)
num_cols = [x for x in X3.columns if X3[x].dtype != 'object']
cat_cols = [x for x in X3.columns if X3[x].dtype == 'object']

num_pipeline = Pipeline(steps=[('scale',MinMaxScaler())])
cat_pipeline = Pipeline(steps=[('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

# LogisticRegression
clf = LogisticRegression(random_state=0)
clf_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', clf)
])
lr = clf_pipeline.fit(X3_train, y3_train)
f1_LR_del_missings = cross_val_score(lr, X3_train, y3_train, cv=3, scoring='f1').mean()
print('f1_LR_del_missings :' , f1_LR_del_missings)
acc_LR_del_missings = cross_val_score(lr, X3_train, y3_train, cv=3, scoring='accuracy').mean()
print('acc_LR_del_missings :' , acc_LR_del_missings)

# KNeighborsClassifier
knnf = KNeighborsClassifier(n_neighbors=2)
knn_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', knnf)
])
knn = knn_pipeline.fit(X3_train, y3_train)
f1_KNN_del_missings = cross_val_score(knn, X3_train, y3_train, cv=3, scoring='f1').mean()
print('f1_KNN_del_missings :' , f1_KNN_del_missings)
acc_KNN_del_missings = cross_val_score(knn, X3_train, y3_train, cv=3, scoring='accuracy').mean()
print('acc_KNN_del_missings :', acc_KNN_del_missings)

# LinearSVC
svmf = LinearSVC(random_state=0, tol=1e-5)
svm_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', svmf)
])
svm = svm_pipeline.fit(X3_train, y3_train)
f1_SVM_del_missings = cross_val_score(svm, X3_train, y3_train, cv=3, scoring='f1').mean()
print('f1_SVM_del_missings :' , f1_SVM_del_missings)
acc_SVM_del_missings = cross_val_score(svm, X3_train, y3_train, cv=3, scoring='accuracy').mean()
print('acc_SVM_del_missings :' , acc_SVM_del_missings)

### Ваш код ###
# f1_LR_del_missings = 0
# acc_LR_del_missings = 0
# f1_KNN_del_missings = 0
# acc_KNN_del_missings = 0
# f1_SVM_del_missings = 0
# acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

f1_LR_del_missings : 0.9043543031430362
acc_LR_del_missings : 0.8499730484279429
f1_KNN_del_missings : 0.8423445665914194
acc_KNN_del_missings : 0.7758553744374281
f1_SVM_del_missings : 0.9060887807197108
acc_SVM_del_missings : 0.8524555994661561


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [46]:
num_pipe = Pipeline(steps=[('scale',MinMaxScaler())])
cat_pipe = Pipeline(steps=[('Simpl',SimpleImputer(missing_values='?', strategy= 'most_frequent')),
                           ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipe,num_cols),
    ('cat_pipeline',cat_pipe,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

# RandomForestClassifier
rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', rfc)
])
rf = rfc_pipeline.fit(X2_train, y2_train)
f1_RF = cross_val_score(rf, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_LR:' ,f1_RF)
acc_RF = cross_val_score(rf, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_LR:' ,acc_RF)

# GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gbc_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', gbc)
])
gb = gbc_pipeline.fit(X2_train, y2_train)
f1_GB = cross_val_score(gb, X2_train, y2_train, cv=3, scoring='f1').mean()
print('f1_GB:' , f1_GB)
acc_GB = cross_val_score(gb, X2_train, y2_train, cv=3, scoring='accuracy').mean()
print('acc_GB:', acc_GB)
### Ваш код ###

# f1_RF = 0
# acc_RF = 0
# f1_GB = 0
# acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

f1_LR: 0.8672622617345441
acc_LR: 0.7670002778472068
f1_GB: 0.9113373849727745
acc_GB: 0.8605686494817973


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [92]:
from sklearn.preprocessing import StandardScaler

def func_sel(num_pipeline, num_cols, cat_pipeline, cat_cols, X2_trn, y2_trn):
    facc = []
    ff1 = []
    def calc_metric(f_cls, num_pipe, num_cols, cat_pipe, cat_cols, X2_trn, y2_trn):

        col_trans = ColumnTransformer(transformers=[
            ('num_pipeline',num_pipe, num_cols),
            ('cat_pipeline',cat_pipe, cat_cols)
            ],
            remainder='drop',
            n_jobs=-1)
   
        clf_pipeline = Pipeline(steps=[
            ('col_trans', col_trans),
            ('model', f_cls)
            ])
        clr = clf_pipeline.fit(X2_trn, y2_trn)
        f1_m = cross_val_score(clr, X2_trn, y2_trn, cv=3, scoring='f1').mean()
        acc_m = cross_val_score(clr, X2_trn, y2_trn, cv=3, scoring='accuracy').mean()
        return acc_m, f1_m
        
    clf = LogisticRegression(random_state=0)
    a,f = calc_metric(clf,num_pipeline,num_cols,cat_pipeline,cat_cols,X2_trn, y2_trn)
    facc.append(a)
    ff1.append(f)

    clf = KNeighborsClassifier(n_neighbors=2)
    a,f = calc_metric(clf,num_pipeline,num_cols,cat_pipeline,cat_cols,X2_trn, y2_trn)
    facc.append(a)
    ff1.append(f)

    clf = LinearSVC(random_state=0, tol=1e-5)
    a,f = calc_metric(clf,num_pipeline,num_cols,cat_pipeline,cat_cols,X2_trn, y2_trn)
    facc.append(a)
    ff1.append(f)

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    a,f = calc_metric(clf,num_pipeline,num_cols,cat_pipeline,cat_cols,X2_trn, y2_trn)
    facc.append(a)
    ff1.append(f)

    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
    a,f = calc_metric(clf,num_pipeline,num_cols,cat_pipeline,cat_cols,X2_trn, y2_trn)
    facc.append(a)
    ff1.append(f)
    return facc, ff1

acc = []
f1 = []
num_pipeline = Pipeline(steps=[('scale',StandardScaler())])
cat_pipeline = Pipeline(steps=[('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])
a1,f11 = func_sel(num_pipeline ,num_cols, cat_pipeline, cat_cols, X2_train, y2_train)
acc.extend(a1)
f1.extend(f11)
print(a1,f11)

num_pipeline = Pipeline(steps=[('scale',StandardScaler())])
cat_pipeline = Pipeline(steps=[('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])
a1,f11 = func_sel(num_pipeline ,num_cols, cat_pipeline, cat_cols, X3_train, y3_train)
acc.extend(a1)
f1.extend(f11)
print(a1,f11)

cat_pipeline = Pipeline(steps=[('Simpl',SimpleImputer(missing_values='?', strategy= 'most_frequent')),
                               ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])
a1,f11 = func_sel(num_pipeline ,num_cols, cat_pipeline, cat_cols, X2_train, y2_train)
acc.extend(a1)
f1.extend(f11)
print(a1,f11)

cat_pipeline = Pipeline(steps=[('Simpl',SimpleImputer(missing_values='?', strategy= 'constant')),
                               ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])
a1,f11 = func_sel(num_pipeline ,num_cols, cat_pipeline, cat_cols, X2_train, y2_train)
acc.extend(a1)
f1.extend(f11)
print(a1,f11)

num_pipeline = Pipeline(steps=[('scale',MinMaxScaler())])
cat_pipeline = Pipeline(steps=[('Simpl',SimpleImputer(missing_values='?', strategy= 'constant')),
                               ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse=False))])
a1,f11 = func_sel(num_pipeline ,num_cols, cat_pipeline, cat_cols, X2_train, y2_train)
acc.extend(a1)
f1.extend(f11)
print(a1,f11)
df = pd.DataFrame({'acc': acc, 'f1': f1})
print('Best acc :', df['acc'].max(),'Best f1 :', df['f1'].max())

[0.8522252456609462, 0.7815114396808255, 0.852634750000393, 0.7611649991904311, 0.8614644100128744] [0.9056421050971476, 0.8463364901385405, 0.9061949316167303, 0.8643880603972232, 0.9121120187970374]
[0.8522252456609462, 0.7815114396808255, 0.852634750000393, 0.7611649991904311, 0.8614644100128744] [0.9056421050971476, 0.8463364901385405, 0.9061949316167303, 0.8643880603972232, 0.9121120187970374]
[0.8515086671036767, 0.7803085813522473, 0.8517390012591548, 0.7670002778472068, 0.8605686494817973] [0.9052980036814322, 0.8454494211400091, 0.9057448361338669, 0.8672622617345441, 0.9113373849727745]
[0.8522252456609462, 0.7815114396808255, 0.852634750000393, 0.7632635826805693, 0.8614644100128744] [0.9056421050971476, 0.8463364901385405, 0.9061949316167303, 0.8654198859827654, 0.9121120187970374]
[0.8499730484279429, 0.7758553744374281, 0.8524555994661561, 0.7632635826805693, 0.8614644100128744] [0.9043543031430362, 0.8423445665914194, 0.9060887807197108, 0.8654198859827654, 0.91211201879

In [104]:
le = LabelEncoder()
data_le=data2.copy()
data_le[cat_cols] = data_le[cat_cols].apply(lambda col: le.fit_transform(col))
Xle = data_le.drop('class',axis=1)
yle = data_le['class']
Xle_train, Xle_test, yle_train, yle_test = train_test_split(Xle, yle, test_size=0.2, random_state=42)
Xle_train.shape, Xle_test.shape, yle_train.shape, yle_test.shape

((39073, 14), (9769, 14), (39073,), (9769,))

In [105]:
min_max_scaler = MinMaxScaler()
Xle_train_smm = min_max_scaler.fit_transform(Xle_train)
Xle_test_smm = min_max_scaler.transform(Xle_test)

st_scaler = StandardScaler()
Xle_train_sts = st_scaler.fit_transform(Xle_train)
Xle_test_sts = st_scaler.transform(Xle_test)

acc_le = []
f1_le = []

# LogisticRegression
lr_le = LogisticRegression(random_state=0)
lr_le = lr_le.fit(Xle_train_smm, yle_train)
acc_LRle =cross_val_score(lr_le, Xle_train_smm, yle_train, cv=3, scoring='accuracy').mean()
f1_LRle = cross_val_score(lr_le, Xle_train_smm, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_LRle)
f1_le.append(f1_LRle)

# KNeighborsClassifier
knc_le = KNeighborsClassifier(n_neighbors=2)
knc_le = knc_le.fit(Xle_train_smm, yle_train)
acc_KNNle = cross_val_score(knc_le, Xle_train_smm, yle_train, cv=3, scoring='accuracy').mean()
f1_KNNle = cross_val_score(knc_le, Xle_train_smm, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_KNNle)
f1_le.append(f1_KNNle)

# LinearSVC
lscv_le = LinearSVC(random_state=0, tol=1e-5)
lscv_le = lscv_le.fit(Xle_train_smm, yle_train)
acc_SVMle = cross_val_score(lscv_le, Xle_train_smm, yle_train, cv=3, scoring='accuracy').mean()
f1_SVMle = cross_val_score(lscv_le, Xle_train_smm, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_SVMle)
f1_le.append(f1_SVMle)

# RandomForestClassifier
rfc_le = RandomForestClassifier(max_depth=2, random_state=0)
rfc_le = rfc_le.fit(Xle_train_smm, yle_train)
f1_RFle = cross_val_score(rfc_le, Xle_train_smm, yle_train, y2_train, cv=3, scoring='f1').mean()
acc_RFle = cross_val_score(rfc_le, Xle_train_smm, yle_train, cv=3, scoring='accuracy').mean()
acc_le.append(acc_RFle)
f1_le.append(f1_RFle)

# GradientBoostingClassifier
gbc_le = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gbc_le = gbc_le.fit(Xle_train_smm, yle_train)
f1_GBle = cross_val_score(gbc_le, Xle_train_smm, yle_train, cv=3, scoring='f1').mean()
acc_GBle = cross_val_score(gbc_le, Xle_train_smm, yle_train, cv=3, scoring='accuracy').mean()
acc_le.append(acc_GBle)
f1_le.append(f1_GBle)

# LogisticRegression
lr_le = LogisticRegression(random_state=0)
lr_le = lr_le.fit(Xle_train_sts, yle_train)
acc_LRle =cross_val_score(lr_le, Xle_train_sts, yle_train, cv=3, scoring='accuracy').mean()
f1_LRle = cross_val_score(lr_le, Xle_train_sts, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_LRle)
f1_le.append(f1_LRle)

# KNeighborsClassifier
knc_le = KNeighborsClassifier(n_neighbors=2)
knc_le = knc_le.fit(Xle_train_sts, yle_train)
acc_KNNle = cross_val_score(knc_le, Xle_train_sts, yle_train, cv=3, scoring='accuracy').mean()
f1_KNNle = cross_val_score(knc_le, Xle_train_sts, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_KNNle)
f1_le.append(f1_KNNle)

# LinearSVC
lscv_le = LinearSVC(random_state=0, tol=1e-5)
lscv_le = lscv_le.fit(Xle_train_sts, yle_train)
acc_SVMle = cross_val_score(lscv_le, Xle_train_sts, yle_train, cv=3, scoring='accuracy').mean()
f1_SVMle = cross_val_score(lscv_le, Xle_train_sts, yle_train, cv=3, scoring='f1').mean()
acc_le.append(acc_SVMle)
f1_le.append(f1_SVMle)

# RandomForestClassifier
rfc_le = RandomForestClassifier(max_depth=2, random_state=0)
rfc_le = rfc_le.fit(Xle_train_sts, yle_train)
f1_RFle = cross_val_score(rfc_le, Xle_train_sts, yle_train, y2_train, cv=3, scoring='f1').mean()
acc_RFle = cross_val_score(rfc_le, Xle_train_sts, yle_train, cv=3, scoring='accuracy').mean()
acc_le.append(acc_RFle)
f1_le.append(f1_RFle)

# GradientBoostingClassifier
gbc_le = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gbc_le = gbc_le.fit(Xle_train_sts, yle_train)
f1_GBle = cross_val_score(gbc_le, Xle_train_sts, yle_train, cv=3, scoring='f1').mean()
acc_GBle = cross_val_score(gbc_le, Xle_train_sts, yle_train, cv=3, scoring='accuracy').mean()
acc_le.append(acc_GBle)
f1_le.append(f1_GBle)

print(acc_le)
print(f1_le)
df = pd.DataFrame({'acc': acc_le, 'f1': f1_le})
print('Best acc :', df['acc'].max(),'Best f1 :', df['f1'].max())

### Ваш код ###
best_params = {'Scaler': 'MinMaxScaler()/StandardScaler()', 'Encoder': 'LabelEncoder()', 'Model': 'GradientBoostingClassifier',
               'param' : 'n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0'}

print('Параметры лучшей модели:\n', best_params)
f1_best = df['f1'].max()
acc_best = df['acc'].max()
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

[0.8229211978948063, 0.7744477678690731, 0.82432882214792, 0.7940009349342363, 0.8658152359303991, 0.8247638888233898, 0.7784147696815643, 0.8242520368911923, 0.7940009349342363, 0.8658152359303991]
[0.8903607980949834, 0.8414820157616671, 0.8921639102703823, 0.8807717479742436, 0.914670468559741, 0.891184488142381, 0.8441908575574889, 0.8920775589390638, 0.8807717479742436, 0.914670468559741]
Best acc : 0.8658152359303991 Best f1 : 0.914670468559741
Параметры лучшей модели:
 {'Scaler': 'MinMaxScaler()/StandardScaler()', 'Encoder': 'LabelEncoder()', 'Model': 'GradientBoostingClassifier', 'param': 'n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0'}


In [101]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904354,0.849973
2,KNeighborsClassifier,task13,0.842345,0.775855
3,LinearSVC,task13,0.906089,0.852456
4,LogisticRegression_impute,task14,0.904615,0.850203
5,KNeighborsClassifier_impute,task14,0.841426,0.774525
6,LinearSVC_impute,task14,0.905589,0.851457
7,LogisticRegression_delete_missings,task15,0.904354,0.849973
8,KNeighborsClassifier_delete_missings,task15,0.842345,0.775855
9,LinearSVC_delete_missings,task15,0.906089,0.852456


In [100]:
results_classification['accuracy'].max()

0.8658152359303991